In [4]:
!nvidia-smi


Wed Sep 18 00:03:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
%%writefile exercicio1.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <iostream>
#include <fstream>
#include <chrono>

int main() {
    // Variáveis para medir o tempo
    std::chrono::steady_clock::time_point start, end;
    std::chrono::duration<double> elapsed;

    // Abrindo o arquivo
    std::ifstream file("stocks-google.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    // Vetor para armazenar os preços na CPU
    thrust::host_vector<double> host_prices;

    // Lendo os dados e armazenando no host_vector
    double price;
    start = std::chrono::steady_clock::now();
    while (file >> price) {
        host_prices.push_back(price);
    }
    end = std::chrono::steady_clock::now();
    elapsed = end - start;
    std::cout << "Tempo de LEITURA (em segundos): " << elapsed.count() << std::endl;

    file.close();

    // Transferindo os dados para a GPU
    start = std::chrono::steady_clock::now();
    thrust::device_vector<double> device_prices = host_prices;
    end = std::chrono::steady_clock::now();
    elapsed = end - start;
    std::cout << "Tempo de ALOCAÇÃO E CÓPIA (em segundos): " << elapsed.count() << std::endl;

    return 0;
}


Writing exercicio1.cu


In [6]:
!nvcc -arch=sm_75 -std=c++14 exercicio1.cu -o exercicio1


In [7]:
!./exercicio1


Tempo de LEITURA (em segundos): 0.00309812
Tempo de ALOCAÇÃO E CÓPIA (em segundos): 0.110018


In [8]:
%%writefile exercicio2.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <thrust/extrema.h>
#include <iostream>
#include <fstream>
#include <chrono>

int main() {
    // Variáveis para medir o tempo
    std::chrono::steady_clock::time_point start, end;
    std::chrono::duration<double> elapsed;

    // Abrindo o arquivo
    std::ifstream file("stocks-google.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    // Vetor para armazenar os preços na CPU
    thrust::host_vector<double> host_prices;

    // Lendo os dados e armazenando no host_vector
    double price;
    start = std::chrono::steady_clock::now();
    while (file >> price) {
        host_prices.push_back(price);
    }
    end = std::chrono::steady_clock::now();
    elapsed = end - start;
    std::cout << "Tempo de LEITURA (em segundos): " << elapsed.count() << std::endl;

    file.close();

    // Transferindo os dados para a GPU
    start = std::chrono::steady_clock::now();
    thrust::device_vector<double> device_prices = host_prices;
    end = std::chrono::steady_clock::now();
    elapsed = end - start;
    std::cout << "Tempo de ALOCAÇÃO E CÓPIA (em segundos): " << elapsed.count() << std::endl;

    // Número total de elementos
    size_t N = device_prices.size();

    // Verificando se há pelo menos 365 dias de dados
    if (N < 365) {
        std::cerr << "Dados insuficientes para os cálculos do último ano." << std::endl;
        return 1;
    }

    // Calculando a soma total dos preços
    double total_sum = thrust::reduce(device_prices.begin(), device_prices.end(), 0.0, thrust::plus<double>());
    double total_mean = total_sum / N;
    std::cout << "Preço médio nos últimos 10 anos: " << total_mean << std::endl;

    // Calculando a soma dos últimos 365 dias
    thrust::device_vector<double>::iterator last_year_start = device_prices.end() - 365;
    double last_year_sum = thrust::reduce(last_year_start, device_prices.end(), 0.0, thrust::plus<double>());
    double last_year_mean = last_year_sum / 365;
    std::cout << "Preço médio nos últimos 365 dias: " << last_year_mean << std::endl;

    // Encontrando o preço máximo e mínimo no período total
    thrust::device_vector<double>::iterator max_iter = thrust::max_element(device_prices.begin(), device_prices.end());
    thrust::device_vector<double>::iterator min_iter = thrust::min_element(device_prices.begin(), device_prices.end());
    std::cout << "Preço máximo nos últimos 10 anos: " << *max_iter << std::endl;
    std::cout << "Preço mínimo nos últimos 10 anos: " << *min_iter << std::endl;

    // Encontrando o preço máximo e mínimo nos últimos 365 dias
    thrust::device_vector<double>::iterator max_iter_last_year = thrust::max_element(last_year_start, device_prices.end());
    thrust::device_vector<double>::iterator min_iter_last_year = thrust::min_element(last_year_start, device_prices.end());
    std::cout << "Preço máximo nos últimos 365 dias: " << *max_iter_last_year << std::endl;
    std::cout << "Preço mínimo nos últimos 365 dias: " << *min_iter_last_year << std::endl;

    return 0;
}


Writing exercicio2.cu


In [9]:
!nvcc -arch=sm_75 -std=c++14 exercicio2.cu -o exercicio2


In [10]:
!./exercicio2


Tempo de LEITURA (em segundos): 0.00375821
Tempo de ALOCAÇÃO E CÓPIA (em segundos): 0.106951
Preço médio nos últimos 10 anos: 1580.08
Preço médio nos últimos 365 dias: 1615.81
Preço máximo nos últimos 10 anos: 2200
Preço mínimo nos últimos 10 anos: 1015.24
Preço máximo nos últimos 365 dias: 2200
Preço mínimo nos últimos 365 dias: 1060.5


In [11]:
%%writefile exercicio3.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <iostream>
#include <fstream>
#include <sstream>

int main() {
    // Abrindo o arquivo
    std::ifstream file("stocks2.txt");
    if (!file.is_open()) {
        std::cerr << "Erro ao abrir o arquivo stocks2.txt" << std::endl;
        return 1;
    }

    // Vetores para armazenar os preços na CPU
    thrust::host_vector<double> host_AAPL_prices;
    thrust::host_vector<double> host_MSFT_prices;

    // Lendo os dados e armazenando nos host_vectors
    std::string line;
    while (std::getline(file, line)) {
        std::stringstream ss(line);
        std::string value;
        double price_AAPL, price_MSFT;

        // Lendo o preço da Apple
        if (std::getline(ss, value, ',')) {
            price_AAPL = std::stod(value);
            host_AAPL_prices.push_back(price_AAPL);
        } else {
            std::cerr << "Erro ao ler o preço da Apple." << std::endl;
            return 1;
        }

        // Lendo o preço da Microsoft
        if (std::getline(ss, value, ',')) {
            price_MSFT = std::stod(value);
            host_MSFT_prices.push_back(price_MSFT);
        } else {
            std::cerr << "Erro ao ler o preço da Microsoft." << std::endl;
            return 1;
        }
    }

    file.close();

    // Verificando se os vetores têm o mesmo tamanho
    if (host_AAPL_prices.size() != host_MSFT_prices.size()) {
        std::cerr << "Os vetores de preços têm tamanhos diferentes." << std::endl;
        return 1;
    }

    // Transferindo os dados para a GPU
    thrust::device_vector<double> device_AAPL_prices = host_AAPL_prices;
    thrust::device_vector<double> device_MSFT_prices = host_MSFT_prices;

    // Vetor para armazenar as diferenças
    thrust::device_vector<double> price_differences(device_AAPL_prices.size());

    // Calculando a diferença ponto a ponto
    thrust::transform(device_AAPL_prices.begin(), device_AAPL_prices.end(),
                      device_MSFT_prices.begin(),
                      price_differences.begin(),
                      thrust::minus<double>());

    // Calculando a média das diferenças
    double total_difference = thrust::reduce(price_differences.begin(), price_differences.end(), 0.0, thrust::plus<double>());
    double mean_difference = total_difference / price_differences.size();

    // Interpretando o resultado
    if (mean_difference > 0) {
        std::cout << "Em média, a Apple teve um preço " << mean_difference << " dólares maior que a Microsoft." << std::endl;
    } else if (mean_difference < 0) {
        std::cout << "Em média, a Microsoft teve um preço " << -mean_difference << " dólares maior que a Apple." << std::endl;
    } else {
        std::cout << "Em média, as duas empresas tiveram o mesmo preço." << std::endl;
    }

    return 0;
}


Writing exercicio3.cu


In [12]:
!nvcc -arch=sm_75 -std=c++14 exercicio3.cu -o exercicio3


In [14]:
!./exercicio3


Em média, a Microsoft teve um preço 265.232 dólares maior que a Apple.
